In [ ]:
import pandas as pd
import geopandas as gpd
import os

In [ ]:
# Open the data on the wave fetchs

fetch_length = gpd.read_file('SHP/Fetch_length.shp').drop(columns=['Distance', 'OBJECTID', 'X', 'Y', 'geometry'])    # Open the fetch length dataset

fetch_length['Bearing'] = fetch_length['Bearing'].replace({180: 'S', 135: 'SE', 157.5: 'SSE', 202.5: 'SSW', 225: 'SW'})   # Rename wind directions from degrees to rhumbs

fetch_length['Shape_Leng'] = fetch_length['Shape_Leng']/1000    # Convert distances from meters to kilometers

In [ ]:
# Calculate the wind wave effect for one transect for one year

def wwe_year_mi(TransectID, meteo_df):
    
    fetch_length1 = fetch_length[(fetch_length['TransectID'] == TransectID)]
    
    grad_cols = ['WSW', 'SW', 'SSW', 'S','SSE', 'SE']    # Create a list with the wind rhumbs
    grad_index = ['5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '40-45', '45-50', '>50']    # Create a list with the wind gradations
    Gradations = pd.DataFrame(columns = grad_cols, index = grad_index)    # Construct a dataframe to calculate a frequency of winds of different gradations of different rhumbs
    Av_wind_speed = Gradations.copy()    # Construct a dataframe to calculate an average speed of winds of different gradations of different rhumbs
   
    # Iterate the rhumbs and the gradations and calculate the frequency and the average speed for them
    for rhumb in grad_cols:
        for speed in grad_index[:-1]:
            lb = int(speed.split('-')[0])
            rb = int(speed.split('-')[1])
            Gradations[rhumb][speed] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed'] > lb) & (meteo_df['wind_speed'] <= rb)].count()
            Av_wind_speed[rhumb][speed] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed'] > lb) & (meteo_df['wind_speed'] <= rb)].mean()

        Gradations[rhumb]['>50'] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed']>50)].count()
        Av_wind_speed[rhumb]['>50'] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed']>50)].mean()
 
    v3 = (Av_wind_speed.fillna(0))**3    # Raise to the 3rd power the wind speed for not null values
   
    wwe = Gradations.copy()    # Create a dataframe for wind-wave effect values
    
    # Iterate the rhumbs and calculate the wind-wave effect for them
    for i, rhumb in wwe.items():
        if rhumb.name in fetch_length1['Bearing'].unique():
            for j in range(0, len(rhumb)):
                wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_length1[(fetch_length1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
        else:
            wwe[rhumb.name] = 0        
    
    return wwe.values.sum()

In [ ]:
# Open the files with weather station data for different years

folder = 'CSV/years'
filelist = os.listdir(folder)

In [ ]:
# Create a dataframe for any year dataset

def df_making(file, folder):
    meteo_df = pd.read_csv(os.path.join(folder, file))
    meteo_df = meteo_df.drop(meteo_df[meteo_df['wind_speed'] == '---'].index)
    meteo_df['wind_speed'] = meteo_df['wind_speed'].astype(float)
    return meteo_df

In [ ]:
# Construct a dataframe for all the years

wwe_all = pd.DataFrame(index = fetch_length['TransectID'].unique(), columns=[filelist])

In [ ]:
# Iterate files with different years, calculate a WWE for every transect every year and fill in the resulted dataframe

for file in filelist:
    meteo_df = df_making(file, folder)
    wwe_year = pd.DataFrame(index = fetch_length['TransectID'].unique(), columns=[file])
    for TransectID in fetch_length['TransectID'].unique():
        transect_wwe = wwe_year_mi(TransectID, meteo_df)
        wwe_year[file][TransectID] = transect_wwe
    wwe_all[file] = wwe_year[file]  

In [ ]:
# Export the resulted dataframe to csv

wwe_all.to_csv('CSV/wwe_all_year_transects.csv')